## Dowloading the dataset

In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

# Reading the dataset

In [ ]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")
with open(fname) as f:
    data = f.read()

In [ ]:
lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)

## Separating the target feature and the input features


In [ ]:
import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header)- 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    temperature[i] =values[1]
    raw_data[i, :] = values[:]

# Plotting the target feature(temperature)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(temperature)), temperature)

In [ ]:
plt.plot(range(1440), temperature[:1440])

In [ ]:
print(temperature)

# Splitting the dataset into train, Val and text

In [ ]:
#splitting data into train , val, text
train_samples = int(len(raw_data) * 0.5)
val_samples = int(len(raw_data) * 0.25)
test_samples = int(len(raw_data)-(train_samples + val_samples))

# Normalizing the dataset

In [ ]:
#normalizing the data
mean = raw_data[:train_samples].mean(axis = 0)
raw_data -= mean
std = raw_data[:train_samples].std(axis = 0)
raw_data /= std

In [ ]:
#timeseries dataset from array
import tensorflow as tf
from tensorflow import keras

# Utilizing the timeseries_dataset_from_array to chuck the dataset set into squence of windows

In [ ]:
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256
train_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets = temperature[delay:],
    sampling_rate =  sampling_rate,
    sequence_length = sequence_length,
    batch_size = batch_size,
    shuffle = True,
    start_index = 0,
    end_index = train_samples


)

In [ ]:
val_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets = temperature[delay:],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    batch_size = batch_size,
    shuffle = True,
    start_index = train_samples,
    end_index = train_samples + val_samples
)

In [ ]:
test_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets = temperature[delay:],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    batch_size = batch_size,
    shuffle = True,
    start_index=  train_samples + val_samples

)

In [ ]:
for samples , targets in train_dataset:
   print(f"sample shape is: {samples.shape}")
   print(f"Target shape is: {targets.shape}")
   break

# Setting a common baseline to measure our models performance against it

In [ ]:
# setting a common baseline
def evaluate_naive_method(dataset):
    total_abs_err = 0
    samples_seen = 0
    for samples, targets in dataset:
        preds = samples[:, -1, 1]* std[1] + mean[1]
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err/ samples_seen

print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}")
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}")

## Building the model using LSMT

In [ ]:
# building the model using LSMT
from tensorflow.keras import layers
inputs = keras.Input(shape = (sequence_length, raw_data.shape[-1]))
x = layers.LSTM(16)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs = inputs, outputs = outputs)

In [ ]:
model.compile(optimizer = "rmsprop",
             loss = "mse",
             metrics = ["mae"])

In [ ]:
callbacks = keras.callbacks.ModelCheckpoint(
    filepath = "timeseries_lstm.keras",
    save_best_only = True,
    monitor = "mae"
)

In [ ]:
model.fit(train_dataset,
         epochs = 10,
         callbacks = callbacks,
         validation_data = val_dataset)

In [ ]:
history = keras.model.load_model("timeseries_lstm.keras")

In [ ]:
print(f"Validation loss is :{history["val_loss"]}")
print(f"Validation MAE is :{history["val_mae"]}")

In [ ]:
print(f"Test mae is : {history.evaluate(test_dataset[1]:.2f)}")